In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

!git clone https://github.com/devkotasawal1/Poem-Generator

Mounted at /content/drive


In [2]:
import numpy as np
from tqdm import tqdm

In [3]:
PATH =  "/content/drive/MyDrive/word2vec_models/nepali_embeddings_word2vec.txt"

In [4]:
from gensim.models import KeyedVectors

In [5]:
model_pretrained = KeyedVectors.load_word2vec_format(PATH, binary=False)
model = model_pretrained

In [6]:
model_pretrained.most_similar_cosmul(positive = ['केटी', 'राजा'], negative = ['केटा'])

[('मुगल', 0.8069719672203064),
 ('महारानी', 0.7991781234741211),
 ('राजाहरू', 0.799119770526886),
 ('राजगद्दी', 0.7980359196662903),
 ('अधिराजकुमार', 0.7924796938896179),
 ('राजदरबार', 0.7917264103889465),
 ('महाराजा', 0.7911354303359985),
 ('सम्राट्', 0.7840874791145325),
 ('राजामहाराजा', 0.7802377343177795),
 ('चन्द्रशम्शेर', 0.7725237607955933)]

In [7]:
word_vectors = model.wv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [8]:
print("Number of word vectors:{}".format(len(word_vectors.vocab)))

Number of word vectors:515851


In [9]:
nb_words = len(word_vectors.vocab)
WV_DIM = word_vectors.vector_size

In [10]:
nb_words, WV_DIM

(515851, 300)

In [11]:
wv_matrix = (np.random.rand(nb_words, WV_DIM)-0.5)/5.0

Now use this word2vec values with all the values in the dataset and use the embeddings of the words present in the dataset and leave all the others as zeros

In [229]:
for word, i in word_index.items():
  if i>= nb_words:
    continue
  try:
    embedding_vector = word_vectors[word]
    # words not found in embedding index will be all-zeros
    wv_matrix[i] = embedding_vector
  except:
    pass

In [13]:
path_to_file = "Poem-Generator/lspd.txt"

In [70]:
from nltk.tokenize.api import StringTokenizer
tokenizer = StringTokenizer()

In [71]:
from collections import Counter
vocab = Counter

In [72]:
def text_to_wordlist(text):
  text = tokenizer.tokenize(text)
  vocab.update(text)
  return text 

In [73]:
def process_data(text_sentences):
  sentences = []
  for text in tqdm(text_sentences):
    txt = text_to_wordlist(text)
    sentences.append(txt)
  return sentences

In [107]:
data_file = open(path_to_file, 'r')
file_contents = data_file.read()
sentences_split = file_contents.splitlines()

In [109]:
len(sentences_split)

4091

In [99]:
import nltk
nltk.download('punkt') 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [199]:
vocab = Counter()
def text_to_wordlist(text, lower = False):
    text = word_tokenize(text)
    vocab.update(text)
    return text
def process_comments(list_sentences, lower = False):
    comments = []
    for text in tqdm(list_sentences):
        txt = text_to_wordlist(text, lower = lower)
        comments.append(txt)
    return comments

In [200]:
comments = process_comments(sentences_split)

100%|██████████| 4091/4091 [00:00<00:00, 8799.26it/s]


In [201]:
comments[1]

['मनको', 'वनमा', 'ननिभ्ने', 'गरी', 'विरह', 'जलाई', '!']

In [217]:
vocab = Counter()
words_split = []
for i in range(len(sentences_split)):
#  words = ""
  for idx, word in enumerate(sentences_split[i].split()):
  #  words = words + " " +word
    word = word_tokenize(word)
    vocab.update(word)
    words_split.append(word)

In [218]:
len(vocab)

8572

In [219]:
vocab['मदन'], vocab['मुना']

(28, 35)

In [220]:
model_new = Word2Vec(words_split, size = 100, window = 5, min_count = 5, workers = 16, sg = 0, negative = 5)

In [221]:
model_new.wv.most_similar('मदन')

[('आँसु', 0.28142857551574707),
 ('मजा', 0.27314072847366333),
 ('जीव', 0.2465236335992813),
 ('गोली', 0.23700422048568726),
 ('जलप', 0.23592029511928558),
 ('यसको', 0.2278319150209427),
 ('अघि', 0.22552791237831116),
 ('पवित्र', 0.2194938063621521),
 ('शून्य', 0.21850956976413727),
 ('रेशम', 0.21591463685035706)]

In [209]:
word_vectors = model_new.wv

In [222]:
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 200

In [223]:
from keras.preprocessing.sequence import pad_sequences

In [224]:
word_index = {t[0]:i+1 for i, t in enumerate(vocab.most_common(MAX_NB_WORDS))}

In [225]:
sequences = [[word_index.get(t,0) for t in comment] for comment in comments[:len(sentences_split)]]

In [226]:
len(sequences)

4091

In [228]:
print(sequences[0])
print(comments[0])

[0, 0, 13, 46, 109, 1, 0, 28, 2]
['नछाडी', 'जानोस्', 'हे', 'मेरा', 'प्राण', '!', 'अकेली', 'मलाई', ',']


**FIX THE ABOVE CELL ERROR**

In [230]:
for word, i in word_index.items():
  if i>= nb_words:
    continue
  try:
    embedding_vector = word_vectors[word]
    # words not found in embedding index will be all-zeros
    wv_matrix[i] = embedding_vector
  except:
    pass

In [231]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout,SpatialDropout1D, Bidirectional
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization


In [232]:
wv_layer = Embedding(nb_words,
                     WV_DIM,
                     mask_zero=False,
                     weights=[wv_matrix],
                     input_length=MAX_SEQUENCE_LENGTH,
                     trainable=False)